# Create a nanowire

Create a nanowire from original material by creating a slab (with specific Miller indices) and cutting a cylinder along its Z axis, then, optionally, rotating to align along X axis.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Click “Run” > “Run All” to run all cells. 
1. Scroll down to view results. 

## 1. Prepare the Environment
### 1.1. Set up nanowire parameters

In [ ]:
MILLER_INDICES= (0,0,1)  # Miller indices of the nanowire direction
RADIUS = 5.0 # Wire radius in Angstroms
LENGTH = 30.0 # Wire length in Angstroms
VACUUM = 10.0 # Vacuum thickness on the sides in Angstroms
ALIGN_ALONG_X = True

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip
  
    await micropip.install("mat3ra-api-examples", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages
    await install_packages("")

### 1.3. Get input materials

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())
material = materials[0] 

### 1.4. Preview the material

In [ ]:
from utils.visualize import visualize_materials as visualize
visualize(material, repetitions=[3,3,3], rotation="0x")
visualize(material, repetitions=[3,3,3], rotation="-90x")

## 2. Create nanowire
### 2.1. Create a slab and cut a cylinder

In [ ]:
import math
from mat3ra.made.tools.helpers import create_supercell
from mat3ra.made.tools.modify import filter_by_cylinder, add_vacuum_sides
from mat3ra.made.tools.helpers import create_slab

slab_unit_cell = create_slab(
    crystal=material,
    miller_indices=MILLER_INDICES,
    number_of_layers=1,
    vacuum=0,
    use_orthogonal_c=True,
)

n_for_x = math.ceil(int(2*RADIUS / slab_unit_cell.lattice.a)) + 1
n_for_y = math.ceil(int(2*RADIUS / slab_unit_cell.lattice.b)) + 1
n_for_z = int(LENGTH / slab_unit_cell.lattice.c)
supercell = create_supercell(slab_unit_cell, supercell_matrix=[[n_for_x, 0, 0], [0, n_for_y, 0], [0, 0, n_for_z]])
nanowire_z = filter_by_cylinder(supercell, radius=RADIUS, use_cartesian_coordinates=True)
nanowire_z = add_vacuum_sides(nanowire_z, vacuum=VACUUM, on_x=True, on_y=True)    

### 2.2. Rotate to align along X axis (optional)

In [ ]:
rotate_90_degree_matrix = [[0, 0, 1], [0, 1, 0], [-1, 0, 0]]
nanowire_x = create_supercell(nanowire_z, supercell_matrix=rotate_90_degree_matrix)
nanowire = nanowire_x if ALIGN_ALONG_X else nanowire_z

## 3. Visualize the result

In [ ]:
visualize(nanowire, repetitions=[1, 1, 1], rotation="0x")
visualize(nanowire, repetitions=[1, 1, 1], rotation="-90x")
visualize(nanowire, repetitions=[1, 1, 1], rotation="-90y")

# 4. Pass material to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials
set_materials(nanowire)